In [9]:
import json
import os
import glob
from pprint import pprint, pformat

import pandas as pd

In [2]:
with open(glob.glob(r'C:\TEMP\*.json')[0]) as fd:
    data = json.load(fd)

In [14]:
def get_range_info(ri: dict) -> dict:
    ri_keys = ('name', 'from', 'to', 'subnet', 'isContainer', 'autoAssign', 'lastModified', 'locked')
    d = {key.upper(): ri.get(key, 'UNDEF') for key in ri_keys}
    cp = ri['customProperties']
    cp_keys = {'Status', 'Environment', 'Region', 'Country', 'DNSCode', 'Description', 'SiteName', 'IsTopLevel', 'Function', 'SiteCode', 'Vlan'}    
    d.update({key.upper(): cp.get(key, 'UNDEF') for key in cp_keys})
    return d

MIN_RANGE_COUNT = 1000
result = data['result']
ranges, total_results = result['ranges'], result['totalResults']
assert total_results > MIN_RANGE_COUNT
assert len(ranges) > MIN_RANGE_COUNT

df = pd.DataFrame(map(get_range_info, ranges))

In [22]:
df.groupby(['STATUS', 'LOCKED']).count().NAME

STATUS         LOCKED
Assigned       False         8
Available      False         2
Discovered     False     28280
Logical-group  False      3886
               True          5
Reserved       False     27346
               True         12
UNDEF          False         1
Undiscovered   False         4
Name: NAME, dtype: int64

In [24]:
df[df.STATUS == 'Logical-group']

,NAME,FROM,TO,SUBNET,ISCONTAINER,AUTOASSIGN,LASTMODIFIED,LOCKED,ISTOPLEVEL,REGION,VLAN,ENVIRONMENT,FUNCTION,SITECODE,DESCRIPTION,DNSCODE,STATUS,COUNTRY,SITENAME
2,0.0.0.0/0,0.0.0.0,255.255.255.255,False,True,False,"Sep 6, 2023 10:24:41",False,1,RESERVED,UNDEF,TOP-LEVEL (WAN),TOP-LEVEL (WAN),RESERVED,Infrastructure,RESERVED,Logical-group,RESERVED,RESERVED
5,10.0.0.0/16,10.0.0.0,10.0.255.255,True,False,True,"Sep 6, 2023 10:24:42",False,0,RESERVED,UNDEF,RESERVED,RESERVED,RESERVED,Available,RESERVED,Logical-group,RESERVED,RESERVED
8,10.0.2.0/24,10.0.2.0,10.0.2.255,True,False,False,"Dec 7, 2023 05:59:50",False,0,AMR,test,GPB (General Purpose Building LAN),"CLI (Wired Client data access (laptops, deskto...",OR,"test123,test123,test123,test123,test123,test12...",hf.intel.com,Logical-group,UNITED STATES,"Oregon, Hillsboro/HF - Hawthorn Farm"
11,10.0.121.0/24,10.0.121.0,10.0.121.255,True,False,False,"Sep 6, 2023 10:24:46",False,1,GAR,UNDEF,WAN (Wide Area Network),"INF (Infrastructure (Backbone, Distribution))",HY,SKC India Hydebarad SIP Trunk,wan.intel.com,Logical-group,INDIA,"India, Hyderabad"
15,10.0.192.0/18,10.0.192.0,10.0.255.255,True,False,True,"Sep 6, 2023 10:24:48",False,1,AMR,UNDEF,TOP-LEVEL (WAN),TOP-LEVEL (WAN),SC,Santa Clara Addressing\nNot normal LAN or DC S...,sc.intel.com,Logical-group,UNITED STATES,"California, Santa Clara"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59480,198.175.112.0/21,198.175.112.0,198.175.119.255,False,False,False,"Sep 14, 2023 18:34:15",False,1,RESERVED,UNDEF,TOP-LEVEL (WAN),TOP-LEVEL (WAN),RESERVED,ARIN Direct Assigned Public CIDR Block\n\nNetR...,RESERVED,Logical-group,RESERVED,RESERVED
59481,198.175.112.0/22,198.175.112.0,198.175.115.255,False,False,False,"Sep 14, 2023 18:34:18",False,1,AMR,UNDEF,IES (Internal Enclave Services),SED (Secure Extended DMZ),FM,Folsom Secure Extended DMZ,fm.intel.com,Logical-group,UNITED STATES,"California, Folsom"
59484,198.175.114.0/24,198.175.114.0,198.175.114.255,True,False,False,"Sep 14, 2023 18:34:23",False,1,AMR,UNDEF,ICC (Inter Company Connection),SED (Secure Extended DMZ),FM,FM Secure Extended DMZ - RVPN Replacement 198....,fm.intel.com,Logical-group,UNITED STATES,"California, Folsom"
59488,198.175.114.32/27,198.175.114.32,198.175.114.63,True,False,True,"Sep 14, 2023 18:34:32",False,0,AMR,eth-s4p4c9 VLAN id 960,ICC (Inter Company Connection),"INF (Infrastructure (Backbone, Distribution))",AN,unnamed range,ice.intel.com,Logical-group,UNITED STATES,"Texas, Austin"
